In [ ]:
import feedparser
import requests
import time
import urllib3



RSS_FEEDS = [
    # Tier 1: Tribunais (Fatos Oficiais)
    "https://res.stj.jus.br/hrestp-c-portalp/RSS.xml",
    
    # Tier 2: Mídia Jurídica (Análise/Buzz)
    "https://www.conjur.com.br/rss.xml",
    "https://www.migalhas.com.br/rss/rss.xml",
    
    # Tier 3: Legislativo
    "https://www.camara.leg.br/noticias/rss/ultimas-noticias"
]



def test_feed(url):
    print(f"📡 Testando: {url} ...", end=" ")
    
    try:

        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        
        response = requests.get(
            url,
            timeout=10,
            headers=headers
        )
        
        if response.status_code != 200:
            print(f"❌ FALHA DE HTTP: {response.status_code}")
            return

        # 2. Teste de Parsing XML
        feed = feedparser.parse(response.content)
        
        if feed.bozo:
            print(f"⚠️ XML SUJO (Bozo: {feed.bozo_exception})", end=" ")
            
        if len(feed.entries) > 0:
            return feed.entries
        else:
            print(f"⚠️ VAZIO (0 notícias encontradas)")

    except Exception as e:
        print(f"❌ ERRO CRÍTICO: {str(e)}")



In [10]:
print("--- INICIANDO DIAGNÓSTICO DE RSS ---")
for url in RSS_FEEDS:
    print("==" * 50)
    feed = test_feed(url)
    print(feed)
    print("==" * 50)
    time.sleep(1) # Respeito ao servidor

--- INICIANDO DIAGNÓSTICO DE RSS ---
📡 Testando: https://res.stj.jus.br/hrestp-c-portalp/RSS.xml ... [{'id': 'https://www.stj.jus.br/sites/portalp/Paginas/Comunicacao/Noticias/2025/05122025-Terceira-Turma-transfere-sessao-ordinaria-de-2-de-dezembro-para-o-proximo-dia-9--as-10h.aspx', 'guidislink': True, 'link': 'https://www.stj.jus.br/sites/portalp/Paginas/Comunicacao/Noticias/2025/05122025-Terceira-Turma-transfere-sessao-ordinaria-de-2-de-dezembro-para-o-proximo-dia-9--as-10h.aspx', 'title': 'Terceira Turma transfere sessão ordinária de 2 de dezembro para o próximo dia 9, às 10h', 'title_detail': {'type': 'text/plain', 'language': None, 'base': '', 'value': 'Terceira Turma transfere sessão ordinária de 2 de dezembro para o próximo dia 9, às 10h'}, 'links': [{'rel': 'alternate', 'type': 'text/html', 'href': 'https://www.stj.jus.br/sites/portalp/Paginas/Comunicacao/Noticias/2025/05122025-Terceira-Turma-transfere-sessao-ordinaria-de-2-de-dezembro-para-o-proximo-dia-9--as-10h.aspx'}], 'su

In [ ]:

RSS_FEEDS = [
    # Tier 1: Tribunais (Fatos Oficiais)
    "https://res.stj.jus.br/hrestp-c-portalp/RSS.xml",
    
    # Tier 2: Mídia Jurídica (Análise/Buzz)
    "https://www.conjur.com.br/rss.xml",
    "https://www.migalhas.com.br/rss/rss.xml",
    
    # Tier 3: Legislativo
    "https://www.camara.leg.br/noticias/rss/ultimas-noticias"
]



def fetch_feed_safe(url):
    try:
        # verify=False resolve o problema do STF
        # timeout=10 evita que o robô fique travado se o site cair
        response = requests.get(
            url, 
            verify=False, 
            timeout=10,
            headers={'User-Agent': 'Mozilla/5.0 bot'}
        )
        
        # Parseia o conteúdo
        feed = feedparser.parse(response.content)
        return feed.entries
        
    except Exception as e:
        print(f"Erro ao ler {url}: {e}")
        return []

## Teste Assincrono

In [11]:
import asyncio
import aiohttp
import feedparser
import ssl
from typing import List, Dict

In [12]:
async def fetch_feed(session: aiohttp.ClientSession, url: str) -> List[Dict]:
    """
    Faz o download do XML de forma assíncrona e extrai os dados.
    """
    print(f"🚀 Disparando request para: {url}")
    
    try:
        # O timeout evita que o processo trave se um site demorar
        async with session.get(url, timeout=10) as response:
            if response.status != 200:
                print(f"❌ Erro {response.status} em {url}")
                return []
            
            # Pegamos o conteúdo bruto (XML text)
            xml_content = await response.text()
            
            # O feedparser processa o texto bruto.
            # Nota: feedparser é síncrono, mas é muito rápido para XMLs pequenos.
            feed = feedparser.parse(xml_content)
            
            parsed_items = []
            for entry in feed.entries:
                # Normalização de dados (alguns feeds usam 'summary', outros 'description')
                summary = entry.get('summary', entry.get('description', ''))
                
                item = {
                    "source": feed.feed.get('title', url), # Tenta pegar o nome do site
                    "title": entry.title,
                    "url": entry.link,
                    "summary": summary
                }
                parsed_items.append(item)
                
            return parsed_items

    except Exception as e:
        print(f"⚠️ Falha ao processar {url}: {str(e)}")
        return []

async def get_all_feeds(urls: List[str]):
    """
    Orquestrador que gerencia todas as conexões simultâneas.
    """
    # Configuração SSL para ignorar erros (Caso STF)
    ssl_context = ssl.create_default_context()
    ssl_context.check_hostname = False
    ssl_context.verify_mode = ssl.CERT_NONE

    # O TCPConnector é quem gerencia o pool de conexões
    connector = aiohttp.TCPConnector(ssl=ssl_context)

    async with aiohttp.ClientSession(connector=connector) as session:
        tasks = []
        for url in urls:
            # Cria a tarefa mas não espera ela terminar ainda
            task = fetch_feed(session, url)
            tasks.append(task)
        
        # gather roda tudo em paralelo e espera todos terminarem
        results = await asyncio.gather(*tasks)
        
        # 'results' será uma lista de listas [[noticias_stf], [noticias_stj]...]
        # Vamos "achatar" (flatten) para uma lista única
        flat_results = [item for sublist in results for item in sublist]
        
        return flat_results

In [14]:
import time

# --- BLOCO DE TESTE CORRIGIDO PARA JUPYTER ---

start = time.time() # Usando time padrão para simplificar

# No Jupyter, basta chamar 'await' direto:
all_news = await get_all_feeds(RSS_FEEDS)

end = time.time()

print(f"\n{'='*60}")
print(f"🏁 FINALIZADO EM {end - start:.2f} SEGUNDOS")
print(f"📦 Total de notícias coletadas: {len(all_news)}")
print(f"{'='*60}\n")

# Mostra as 5 primeiras para conferência
for news in all_news[:5]:
    print(f"📰 [{news['source']}] {news['title']}")
    print(f"🔗 {news['url']}")
    print(f"📝 {news['summary'][:300]}\n")

🚀 Disparando request para: https://res.stj.jus.br/hrestp-c-portalp/RSS.xml
🚀 Disparando request para: https://www.conjur.com.br/rss.xml
🚀 Disparando request para: https://www.migalhas.com.br/rss/rss.xml
🚀 Disparando request para: https://www.camara.leg.br/noticias/rss/ultimas-noticias

🏁 FINALIZADO EM 1.12 SEGUNDOS
📦 Total de notícias coletadas: 164

📰 [Noticias do STJ] Terceira Turma transfere sessão ordinária de 2 de dezembro para o próximo dia 9, às 10h
🔗 https://www.stj.jus.br/sites/portalp/Paginas/Comunicacao/Noticias/2025/05122025-Terceira-Turma-transfere-sessao-ordinaria-de-2-de-dezembro-para-o-proximo-dia-9--as-10h.aspx
📝 Terceira Turma transfere sessão ordinária de 2 de dezembro para o próximo dia 9, às 10h

📰 [Noticias do STJ] Informativo destaca data-base para benefícios na execução penal e nulidade de inquirição e interrogatório
🔗 https://www.stj.jus.br/sites/portalp/Paginas/Comunicacao/Noticias/2025/05122025-Informativo-destaca-data-base-para-beneficios-na-execucao-penal-e